<a href="https://colab.research.google.com/github/WadeShadow/nlp2021/blob/main/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

folder = '/content/drive/MyDrive/Colab Notebooks/NLP_labs/'

## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [5]:
RL_file = open(folder+'sentences.txt')
RL_sentences = RL_file.read().split('. ')[:-1]

DARINA_file = open(folder+'100.txt', encoding='windows-1251')
DARINA_sentences = DARINA_file.read().split('. ')[:-1]

max_tokens = 10000

sentences = RL_sentences+DARINA_sentences
tokens_count = 0
for sent in sentences:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(sentences))
avg_tokens

RL_sentences

['Training a virtual agent to outperform human players can teach us how to optimize different processes in a variety of different and exciting subfields',
 'This is what Google DeepMind did with its popular AlphaGo, which beat the strongest Go player in history and scored a goal that was considered impossible at the time.In this article, we will develop an AI agent that is able to learn how to play the popular game Snake from scratch',
 'To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow and PyTorch (both versions are available, you can choose the one you prefer)',
 'This approach consists in the interaction between two components: an environment (the game itself) and an agent (Snake)',
 'The agent collects information about its current state (we will see later what this means) and performs an action accordingly',
 'The environment rewards or punishes the agent based on the performed action',
 'Over time, the agent learns what actions m

In [6]:
sent_df = []

for sent in RL_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Reinforcement Learning'})

for sent in DARINA_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Neural network module for recognizing brain diseases'})

sent_df = pd.DataFrame(data=sent_df, columns=['Sentence', 'Target'])

sent_df['Label'] = sent_df['Target']
sent_df.replace({'Label':{'Reinforcement Learning':0, 'Neural network module for recognizing brain diseases':1}}, inplace=True)
sent_df

,Sentence,Target,Label
0,Training a virtual agent to outperform human p...,Reinforcement Learning,0
1,This is what Google DeepMind did with its popu...,Reinforcement Learning,0
2,"To do it, we implement a Deep Reinforcement Le...",Reinforcement Learning,0
3,This approach consists in the interaction betw...,Reinforcement Learning,0
4,The agent collects information about its curre...,Reinforcement Learning,0
...,...,...,...
211,This includes balancing the prospective benefi...,Neural network module for recognizing brain di...,1
212,"Users report their symptoms into the app, whic...",Neural network module for recognizing brain di...,1
213,Most common users could be people with serious...,Neural network module for recognizing brain di...,1
214,Body scans can spot cancer and vascular diseas...,Neural network module for recognizing brain di...,1


#### Tokenization&Embedding

In [7]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(sent_df['Sentence'])

text_vectorizer(sent_df['Sentence'])

<tf.Tensor: shape=(216, 23), dtype=int64, numpy=
array([[ 108,    6,  340, ...,  120,    4, 1214],
       [  27,    8,   47, ...,    6,   76,    9],
       [   3,  152,   26, ...,  618,   22,  216],
       ...,
       [  86,  314,  342, ...,    7,   64,  126],
       [1387,  794,   20, ..., 1159,    0,    0],
       [ 244,    2,  119, ...,  446,    8,    2]])>

In [8]:
embedding = layers.Embedding(input_dim=max_tokens, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=avg_tokens) 

embedding(text_vectorizer(sent_df['Sentence']))

<tf.Tensor: shape=(216, 23, 128), dtype=float32, numpy=
array([[[ 0.01969576, -0.03363824, -0.01714592, ...,  0.01365869,
          0.03427646,  0.02972415],
        [ 0.03420578, -0.01101417,  0.04510173, ...,  0.03625819,
         -0.02382795,  0.04935024],
        [ 0.0143087 , -0.00144192,  0.02813578, ..., -0.02884343,
          0.04720663,  0.0441626 ],
        ...,
        [-0.04499949,  0.03520257, -0.02459498, ..., -0.02703277,
          0.04701767, -0.01599924],
        [ 0.00321423, -0.00910916, -0.01363962, ..., -0.03995229,
          0.02549719, -0.02849528],
        [ 0.01316449,  0.01070199, -0.02204137, ..., -0.04600224,
          0.04782996,  0.04415653]],

       [[-0.01354518, -0.02763556, -0.02580938, ...,  0.02940794,
          0.02088847, -0.03193023],
        [-0.01119832, -0.01310977, -0.00865763, ..., -0.04889328,
         -0.01259655,  0.011397  ],
        [ 0.00604831,  0.028547  ,  0.04286608, ..., -0.00468997,
         -0.04094514, -0.03548393],
        ...

#### Train/Test Split

In [12]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(sent_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Target,Label
135,"By sifting out healthy patients, the system al...",Neural network module for recognizing brain di...,1
17,"In this example, we don’t know the best action...",Reinforcement Learning,0
13,These approaches solve problems that need to b...,Reinforcement Learning,0
3,This approach consists in the interaction betw...,Reinforcement Learning,0
140,A key difference is that algorithms need a lot...,Neural network module for recognizing brain di...,1
...,...,...,...
203,"For example, if a clinical note includes a lis...",Neural network module for recognizing brain di...,1
188,Disease identification and diagnosis becomes m...,Neural network module for recognizing brain di...,1
64,It’s up to the model to figure out how to perf...,Reinforcement Learning,0
198,The community can help find the best and most ...,Neural network module for recognizing brain di...,1


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: USE та USE-FT, здійснити порівняння моделей

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_NB = Pipeline([
                    ("tfidf", TfidfVectorizer()), 
                    ("clf", MultinomialNB()) 
])

# Fit the pipeline to the training data
model_NB.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_NB.score(test_split['Sentence'], test_split['Label'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 90.91%


### Sentence Encoder

In [33]:
import tensorflow_hub as hub
url="https://tfhub.dev/google/universal-sentence-encoder/4"

sentence_encoder_layer = hub.KerasLayer(url,input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

model_use = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_use.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_use.summary()


Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_8 (Dense)             (None, 64)                32832     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [34]:
model_use_history=model_se.fit(train_split['Sentence'],train_split['Label'],epochs=5,
                        validation_data=(test_split['Sentence'],test_split['Label']))

Epoch 1/5
6/6 [==============================] - 4s 143ms/step - loss: 0.6856 - accuracy: 0.5814 - val_loss: 0.6613 - val_accuracy: 0.8409
Epoch 2/5
6/6 [==============================] - 0s 29ms/step - loss: 0.6433 - accuracy: 0.8779 - val_loss: 0.6209 - val_accuracy: 0.9318
Epoch 3/5
6/6 [==============================] - 0s 32ms/step - loss: 0.6011 - accuracy: 0.9419 - val_loss: 0.5746 - val_accuracy: 0.9318
Epoch 4/5
6/6 [==============================] - 0s 31ms/step - loss: 0.5527 - accuracy: 0.9419 - val_loss: 0.5223 - val_accuracy: 0.9318
Epoch 5/5
6/6 [==============================] - 0s 32ms/step - loss: 0.5011 - accuracy: 0.9419 - val_loss: 0.4659 - val_accuracy: 0.9545


### Sentence Encoder with fine tuning

In [41]:
tf_hub_embedding_layer = hub.KerasLayer(url,input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=True, 
                                        name="USE-FT")

model_use_ft = tf.keras.Sequential([
  tf_hub_embedding_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_use_ft")

# Compile model
model_use_ft.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


model_use_ft.summary()


Model: "model_use_ft"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE-FT (KerasLayer)         (None, 512)               256797824 
                                                                 
 dense_12 (Dense)            (None, 64)                32832     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 256,830,721
Non-trainable params: 0
_________________________________________________________________


In [42]:
model_use_ft_history=model_use_ft.fit(train_split['Sentence'],train_split['Label'],epochs=10,
                        validation_data=(test_split['Sentence'],test_split['Label']))


Epoch 1/10
6/6 [==============================] - 10s 362ms/step - loss: 0.6610 - accuracy: 0.6047 - val_loss: 0.5865 - val_accuracy: 0.9773
Epoch 2/10
6/6 [==============================] - 1s 248ms/step - loss: 0.5339 - accuracy: 0.9419 - val_loss: 0.4574 - val_accuracy: 0.9318
Epoch 3/10
6/6 [==============================] - 1s 252ms/step - loss: 0.3861 - accuracy: 0.9477 - val_loss: 0.3236 - val_accuracy: 0.9545
Epoch 4/10
6/6 [==============================] - 1s 249ms/step - loss: 0.2439 - accuracy: 0.9884 - val_loss: 0.2192 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 1s 249ms/step - loss: 0.1423 - accuracy: 0.9884 - val_loss: 0.1518 - val_accuracy: 0.9773
Epoch 6/10
6/6 [==============================] - 1s 247ms/step - loss: 0.0748 - accuracy: 0.9942 - val_loss: 0.1216 - val_accuracy: 0.9773
Epoch 7/10
6/6 [==============================] - 1s 253ms/step - loss: 0.0411 - accuracy: 1.0000 - val_loss: 0.1005 - val_accuracy: 0.9773
Epoch 8/10
6/6 [===

## Здійснити порівняння ефективності моделей

### NB

In [36]:
calculate_results(test_split['Label'], model_NB.predict(test_split['Sentence']))

{'accuracy': 90.9090909090909,
 'f1': 0.9079425837320575,
 'precision': 0.9225589225589226,
 'recall': 0.9090909090909091}

### USE

In [37]:
model_use_preds=model_use.predict(test_split['Sentence'])
model_use_preds=tf.squeeze(tf.round(model_use_preds))
calculate_results(test_split['Label'],model_use_preds)

{'accuracy': 95.45454545454545,
 'f1': 0.9545454545454546,
 'precision': 0.9584980237154149,
 'recall': 0.9545454545454546}

### USE-FT

In [43]:
model_use_ft_preds=model_use_ft.predict(test_split['Sentence'])
model_use_ft_preds=tf.squeeze(tf.round(model_use_ft_preds))
calculate_results(test_split['Label'],model_use_ft_preds)

{'accuracy': 97.72727272727273,
 'f1': 0.9772844726333099,
 'precision': 0.9783057851239669,
 'recall': 0.9772727272727273}